In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools

In [ ]:
# import the required libraries
from czitools.metadata_tools import czi_metadata as czimd
from czitools.read_tools import read_tools as czird
from czitools.utils import misc
from IPython.display import display, HTML
from pathlib import Path
import os
import dask.array as da
import ipywidgets as widgets

if not IN_COLAB:
    from czitools.napari_tools import napari_tools
    import napari


In [ ]:
czifiles = [("CellDivision_T10_Z15_CH2_DCV_small.czi", r"https://github.com/sebi06/czitools/raw/main/data/CellDivision_T10_Z15_CH2_DCV_small.czi"),
            ("CellDivision_T3_Z5_CH2_X240_Y170.czi", r"https://github.com/sebi06/czitools/raw/main/data/CellDivision_T3_Z5_CH2_X240_Y170.czi"),
            ("WP96_4Pos_B4-10_DAPI.czi", r"https://github.com/sebi06/czitools/raw/main/data/WP96_4Pos_B4-10_DAPI.czi"),
            ("w96_A1+A2.czi", r"https://github.com/sebi06/czitools/raw/main/data/w96_A1%2BA2.czi")]

In [ ]:
wd = widgets.Select(
    options=czifiles,
    value=r"https://github.com/sebi06/czitools/raw/main/data/CellDivision_T3_Z5_CH2_X240_Y170.czi",
    description='CZI Files:',
    layout={'width': 'max-content'}
)

display(wd)

In [ ]:
selected_file = wd.value

print(f"Selected CZI File {selected_file}")

In [ ]:
%%capture
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(selected_file)

# get the CZI metadata dictionary directly from filename
mdict = czimd.create_md_dict_red(mdata, sort=False, remove_none=True)

# convert metadata dictionary to a pandas dataframe
mdframe = misc.md2dataframe(mdict)

In [ ]:
# create a ipywdiget to show the dataframe with the metadata
wd = widgets.Output(layout={"scrollY": "auto", "height": "300px"})

with wd:
    display(HTML(mdframe.to_html()))
display(widgets.VBox(children=[wd]))

In [ ]:
# return array with dimension order STCZYX(A)
array6d, mdata= czird.read_6darray(selected_file, use_dask=True, chunk_zyx=True)

# show dask array structure
if isinstance(array6d, da.Array):
    print(array6d)
else:
    print("Shape:", array6d.shape, "dtype:", array6d.dtype)


In [ ]:
if not IN_COLAB:

    # show array inside napari viewer
    viewer = napari.Viewer()
    layers = napari_tools.show(viewer, array6d, mdata,
                               blending="additive",
                               contrast='from_czi',
                               gamma=0.85,
                               show_metadata="table",
                               name_sliders=True)

In [ ]:
napari.utils.nbscreenshot(viewer)